In [4]:
!pip install pycoingecko
!pip install configparser
!pip install tabulate

## Source : https://github.com/man-c/pycoingecko

CoinGecko API wrapper
PyPi Version GitHub

In [24]:
import time 
import datetime
import pandas as pd
import requests
import configparser
from pycoingecko import CoinGeckoAPI
import json

cg = CoinGeckoAPI()

In [6]:
config = configparser.ConfigParser()
config.read('..\..\config.conf')
config.sections()
config["telegram_RedNavi_bot"]['bot_token']

'247419374:AAGMXRpFeh26gzp4vF-I7VK2M4EEpNONCmU'

In [8]:
coins =['bitcoin', 'bitcoin-cash', 'ethereum', 'litecoin', 'ripple']
currency="MYR"
coin_market=dict()
coin_market_hourly=dict()
for coin in coins:
    print (coin)
    coin_market[coin]=(cg.get_coin_market_chart_by_id(id=coin,vs_currency="MYR",days="90",interval="daily"))
    coin_market_hourly[coin]=(cg.get_coin_market_chart_by_id(id=coin,vs_currency="MYR",days="90",interval="hourly"))

bitcoin
bitcoin-cash
ethereum
litecoin
ripple


In [9]:
def convert_unix(time): #For fuck sake this is annoying To convert time 
    #return datetime.datetime.fromtimestamp(int(str(time)[:10])).strftime('%Y-%m-%d %H:%M:%S')
    return datetime.datetime.fromtimestamp(int(str(time)[:10])).strftime('%d/%m/%y %a %H:%M')
    #return datetime.datetime.fromtimestamp(int(str(time)[:10])).strftime('%Y-%m-%d')

def convert_data_frame(dict_data):
    df =pd.DataFrame(dict_data["prices"])
    df.columns = ['posix', 'price']
    #df.date=df.apply(lambda row : add(row['A'],row['B'], row['C']), axis = 1)vconvert_unix(time)
    df["date"]=df.apply(lambda row : convert_unix(row['posix']), axis = 1)
    price_now=df['price'].values[len(df.price)-1]
    df["diff_between_now"]=(price_now/df['price']-1)*100
    df["change"]= df.price.pct_change() * 100
    
    return df[["date","price","diff_between_now","change"]].round(2)
print(convert_data_frame(coin_market['bitcoin']))


def telegram_bot_sendtext(bot_message):
    bot_token = config["telegram_RedNavi_bot"]['bot_token']
    #bot_chatID = '269721616'562593592
    bot_chatID = config["telegram_RedNavi_bot"]['bot_chatID']
    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message
    response = requests.get(send_text)
    return response.json()

                  date      price  diff_between_now  change
0   16/03/21 Tue 08:00  229499.42            -37.02     NaN
1   17/03/21 Wed 08:00  233799.66            -38.18    1.87
2   18/03/21 Thu 08:00  243230.02            -40.57    4.03
3   19/03/21 Fri 08:00  238147.99            -39.31   -2.09
4   20/03/21 Sat 08:00  239234.23            -39.58    0.46
..                 ...        ...               ...     ...
86  10/06/21 Thu 08:00  154627.13             -6.52   11.84
87  11/06/21 Fri 08:00  152060.07             -4.95   -1.66
88  12/06/21 Sat 08:00  153184.99             -5.64    0.74
89  13/06/21 Sun 08:00  146516.54             -1.35   -4.35
90  13/06/21 Sun 14:42  144540.41              0.00   -1.35

[91 rows x 4 columns]


In [259]:
import json
from tabulate import tabulate
coin_market_hourly=dict()

try:
    input_file=open("previous_price.json","r")
    previous=input_file.read()
    previous_price=json.loads(previous.replace("'",'"'))
except IOError:
    initialize="""{"bitcoin": {"myr": 0}, "litecoin": {"myr": 0}, "ethereum": {"myr": 0}, "ripple": {"myr": 0}, "bitcoin-cash": {"myr": 0}}"""
    previous_price=json.loads(initialize.replace("'",'"'))    
finally:
    input_file.close()

input_file=open("previous_price.json","r")
previous=input_file.read()
previous_price=json.loads(previous.replace("'",'"'))
current_price=cg.get_price(ids=coins, vs_currencies=currency)
#print(current_price)
with open("previous_price.json", "w") as outfile:
    outfile.write(str(current_price).replace("'",'"'))
#print(previous_price)
#print(current_price)
df_prev=pd.DataFrame(previous_price)
df_curr=pd.DataFrame(current_price)
df_new=(df_curr/df_prev-1)*100
df_new_t=df_new.T
df_new_t=df_new_t.round(3)
df_new_t=df_new_t.reset_index()
df_new_t["change"]=df_new_t["myr"]
df_new_t.loc[df_new_t.myr > 0, 'change'] = "🟢"
df_new_t.loc[df_new_t.myr < 0, 'change'] = "🔴"
df_new_t.loc[df_new_t.myr == 0, 'change'] = "🟡"
#test = telegram_bot_sendtext(message)
#print(tabulate(df_new_t.values.tolist(),headers=["Coin","Change","Status"], tablefmt='psql'))
message = tabulate(df_new_t.values.tolist(), tablefmt='simple')
test = telegram_bot_sendtext(message)

In [253]:
test = telegram_bot_sendtext("Testing Telegram bot")
print(test)


{'ok': True, 'result': {'message_id': 2353, 'from': {'id': 247419374, 'is_bot': True, 'first_name': 'Personal_help', 'username': 'RedNavi_bot'}, 'chat': {'id': -562593592, 'title': 'Bot Famliy', 'type': 'group', 'all_members_are_administrators': True}, 'date': 1623576136, 'text': 'Testing Telegram bot'}}


In [48]:
convert_data_frame(coin_market_hourly['bitcoin']).tail()
print(str (convert_data_frame(coin_market_hourly['bitcoin']).tail()[["date","price","diff_between_now"]].to_csv(index=False,sep='|')))
#test = telegram_bot_sendtext((str (convert_data_frame(coin_market_hourly['bitcoin'][[da]]).tail().to_csv(index=False,sep='|'))))

date|price|diff_between_now
21-06-11 08:23|152620.06084973205|-0.7685842137980181
21-06-11 09:17|149745.64957094996|1.1361916615787893
21-06-11 10:07|149717.64615999567|1.1551083251123861
21-06-11 11:09|150061.27526873798|0.9234706847135454
21-06-11 11:31|151447.04715495207|0.0



In [36]:
test_string="""<pre language="c++">code</pre>,"""
test = telegram_bot_sendtext(test_string)

In [240]:
from tabulate import tabulate
df=convert_data_frame(coin_market_hourly['bitcoin']).tail()[["date","price","diff_between_now","diff_previous_day"]]
#print(tabulate(df, headers='keys', tablefmt='psql'))
#print(tabulate(df, headers='keys', tablefmt='fancy_grid'))
print(tabulate(df.values.tolist(), headers=['date',"price","Diff_between","Diff_prev"], tablefmt='psql'))
print(df.values.tolist())
telegram_bot_sendtext(tabulate(df.values.tolist(), headers=['date',"price","Diff_between","Diff_prev"], tablefmt='psql'))

#telegram_bot_sendtext("""<code>inline fixed-width code</code>""")

KeyError: 'bitcoin'

In [1]:
convert_data_frame(coin_market_hourly['bitcoin'])
df_hourly=convert_data_frame(coin_market_hourly['bitcoin'])[["date","change"]]
df_hourly.tail(5)

NameError: name 'convert_data_frame' is not defined